In [3]:
import subprocess
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

flask_process = subprocess.Popen(['python3', 'apps.py'])

print(f"Flask app is running with PID {flask_process.pid}")

Flask app is running with PID 44719


python3: can't open file '/home/ubuntu/code/DE_ML/HITIT_Server/apps.py': [Errno 2] No such file or directory


In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")
model = AutoModelForMaskedLM.from_pretrained("kakaobank/kf-deberta-base")

# Tokenize하기

In [2]:
input_text = "서유준의 매출이 하락했으며 장래가 불투명합니다"
tokens = tokenizer.tokenize(input_text) #['서유','##준','##의','매출','##이','하락','##했', ##으며','장래',...] 이런식으로

inputs = tokenizer(input_text, return_tensors="pt") #카카오 그 모델에 등록된 단어와 매칭시켜서 숫자로 

model_output = model(**inputs)

# [MASK]된 단어 예측하기

In [91]:
# 모델을 사용하여 예측 logits 계산

input_text = "매출이 상승했으며 [MASK]가 유망합니다"
tokens = tokenizer.tokenize(input_text)
print(tokens)

inputs = tokenizer(input_text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# 토큰의 가장 가능성 높은 예측값 가져오기
predicted_token_id = torch.argmax(logits[0, 3]).item()  # [MASK] 위치의 토큰 (여기서 3번째 위치)
predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)

# # [MASK] 토큰을 예측된 토큰으로 대체하여 문장 복원
restored_text = input_text.replace("[MASK]", predicted_token)
print(f"Restored text: {restored_text}")

['매출', '##이', '상승', '##했', '##으며', '[MASK]', '가', '유망', '##합니다']
Restored text: 매출이 상승했으며 상승가 유망합니다


# 감정분석하기

In [2]:
from transformers import pipeline
# 감정 분석 파이프라인 로드
sentiment_model = pipeline(model="WhitePeak/bert-base-cased-Korean-sentiment")

2024-06-20 14:55:20.270620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 14:55:22.347778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [44]:
def get_sentiment(text):
    result = sentiment_model(text)
    for elem in result :
        if elem['label'] == "LABEL_1" :
            elem['label'] = "POSITIVE"
        else :
            elem['label'] = "NEGATIVE"
        elem['score'] = round(elem['score'],5)
    
    # print(f"Sentiment analysis result: {result}")
    return result[0]

# 뉴스데이터 크롤링하기

In [19]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

from pykrx import stock
from pykrx import bond

current_date = datetime.now().strftime("%Y%m%d")
tickers = stock.get_market_ticker_list(current_date)
print(tickers)
# print(current_date_formatted)

['095570', '006840', '027410', '282330', '138930', '001460', '001465', '001040', '079160', '00104K', '000120', '011150', '011155', '001045', '097950', '097955', '000480', '000590', '012030', '016610', '005830', '000990', '139130', '001530', '000210', '000215', '375500', '37550L', '37550K', '007340', '004840', '155660', '069730', '017860', '017940', '365550', '383220', '007700', '114090', '078930', '006360', '001250', '007070', '078935', '012630', '039570', '089470', '294870', '009540', '267250', '267270', '443060', '010620', '322000', '042670', '267260', '329180', '097230', '014790', '003580', '204320', '060980', '011200', '035000', '003560', '175330', '234080', '001060', '001067', '001065', '096760', '105560', '432320', '002380', '344820', '009070', '009440', '119650', '092220', '003620', '016380', '001390', '033180', '015590', '001940', '025000', '092230', '000040', '044450', '030200', '033780', '058850', '058860', '093050', '003550', '034220', '051900', '051905', '373220', '003555',

In [22]:
def naver_news_crawling(keyword_list, day):
    result_object = {}  # 결과를 저장할 객체

    for keyword in keyword_list:
        url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={day}&de={day}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240101to20240101&is_sug_officeid=0&office_category=0&service_area=0"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = []
        for item in soup.select('.news_wrap'):
            press_element = item.select_one('a.info.press')
            # print()
            press = item.select_one('a.info.press').get_text().replace(' 선정', '').strip()
            anchor = item.select_one('a.news_tit')
            title = anchor.get_text().strip()
            url = anchor['href']
            dsc = item.select_one('.news_dsc').get_text().strip()

            result.append({
                'press': press,
                'title': title,
                'url': url,
                'dsc': dsc,
            })

        result_object[keyword] = result  # 키워드를 키로 사용하여 결과 저장
    return result_object  # 객체 반환

In [30]:
keyword_list = ["삼성전자"]
day = "2024.06.20"
results = naver_news_crawling(keyword_list, day)
print(results)
#뉴스통해서 받은 제목과 기사내용을 바탕으로 감정분석하기
for keyword, articles in results.items():
    print(f"Keyword: {keyword}")
    for article in articles:
        
        print(f"Description: {article['dsc']}")
        returned = get_sentiment(article['dsc'])
        
        print("\n------------------------------\n")

{'삼성전자': [{'press': '머니S언론사', 'title': "삼성전자, 보급폰 '갤럭시 A35 5G' 21일 국내 출시", 'url': 'https://www.moneys.co.kr/article/2024062008220012105', 'dsc': "삼성전자가 합리적인 가격에 5G 통신을 지원하는 '갤럭시 A35 5G'를 21일 국내 출시한다. 갤럭시 A35 5G는 국내이동통신 3사와 자급제 모델로 출시된다. 색상은 라이트 블루, 라이트 바이올렛, 블루 블랙의 3가지로 출시되며 가격은 49만9400원이다. 엔터테인먼트 시청에 최적화된 스마트폰으로 168.3mm..."}, {'press': '조선비즈언론사', 'title': '삼성전자, 보급형 스마트폰 ‘갤럭시 A35′ 내일 출시…49만9400원', 'url': 'https://biz.chosun.com/it-science/ict/2024/06/20/RGECIE7ZYZBZNLGZZG3C5RDJFA/?utm_source=naver&utm_medium=original&utm_campaign=biz', 'dsc': '색상은 라이트 블루·라이트 바이올렛·블루 블랙 3가지 가격은 49만9400원 삼성전자가 갤럭시 A 시리즈 보급형 스마트폰 ‘갤럭시 A35 5G’를 21일 국내 출시한다. 이 제품은 지난 3월 전 세계 시장에 먼저 출시된 바 있다. 삼성전자는 20일 이 제품을 국내 이동통신 3사와 자급제 모델로 출시한다고 밝혔다....'}, {'press': '이데일리언론사', 'title': '이걸로 전기료 낮췄다.. 삼성전자가 내놓은 야심작', 'url': 'http://www.edaily.co.kr/news/newspath.asp?newsid=02168086638923688', 'dsc': '삼성전자가 인공지능(AI)과 반도체 소자를 결합해 에너지 효율을 높인 ‘비스포크 AI 하이브리드’ 냉장고를 국내 최초로 출시하며 신시장 개척에 나섰다. 비스포크 AI 하이브리드는 기존 냉장고 냉각 방식에서 벗어나 반도

In [56]:
connection.close()

In [58]:
from mysqlconnect import connect_to_mysql

connection = connect_to_mysql()
cursor = connection.cursor()
cursor.execute("select stock_code,name from stocks_products where (DATE(sentiment_update) IS NULL OR DATE(sentiment_update) <> DATE(NOW()))")

rows = cursor.fetchall()
stocks = [ elem[:2] for elem in rows] #(종목코드, 종목명 가져와보리기) DB에서

current_date = datetime.now().strftime("%Y.%m.%d")

for elem in stocks:
    results = naver_news_crawling([elem[1]],current_date)
    print(f"index : {stocks.index(elem)}/{len(stocks)}, {keyword}")
    try : 
        for keyword, articles in results.items():
            positive, negative = 0, 0
            for article in articles:          
                # print(f"Description: {article['dsc']}")
                returned = get_sentiment(article['dsc'])
                # print(returned)
                if returned['label'] == "NEGATIVE":
                    positive += 1
                else :
                    negative += 1
            # print(f"positive : {positive}, negative : {negative}")
            sentiment = 1 if positive > negative else 0
            
            query = f"""\
            UPDATE stocks_products 
            SET sentiment = {sentiment} ,
            sentiment_update = NOW()
            where stock_code = {elem[0]}
            """
            cursor.execute(query)
            connection.commit()
    except:
        print(f"PASS {keyword}")
        continue
        
connection.close()
# for ticker in stock.get_market_ticker_list():
#         종목 = stock.get_market_ticker_name(ticker)
#         print(종목)

USER: hitit-user, HOST: hitit-db-mydata.c9oy8g6q0v76.ap-northeast-2.rds.amazonaws.com, PORT: 3306, DATABASE: hitit
Connected to MySQL database
index : 0/2670, 유한양행
index : 1/2670, CJ대한통운
index : 2/2670, 하이트진로홀딩스
index : 3/2670, 두산
index : 4/2670, 성창기업지주
index : 5/2670, DL
index : 6/2670, 유유제약
index : 7/2670, 일동홀딩스
index : 8/2670, 한국앤컴퍼니
index : 9/2670, 삼천당제약
index : 10/2670, 기아
index : 11/2670, 대유플러스
index : 12/2670, 노루홀딩스
index : 13/2670, 한화손해보험
index : 14/2670, 삼화페인트
index : 15/2670, 롯데손해보험
index : 16/2670, 대원강업
index : 17/2670, 중앙에너비스
index : 18/2670, CR홀딩스
index : 19/2670, 대동
index : 20/2670, 가온전선
index : 21/2670, 삼일제약
index : 22/2670, 흥국화재
index : 23/2670, CS홀딩스
index : 24/2670, 동아쏘시오홀딩스
index : 25/2670, 천일고속
index : 26/2670, SK하이닉스
index : 27/2670, 영풍
index : 28/2670, LS네트웍스
index : 29/2670, 유수홀딩스
index : 30/2670, 현대건설
index : 31/2670, 이화산업
index : 32/2670, 삼성화재
index : 33/2670, 화천기공
index : 34/2670, 강남제비스코
index : 35/2670, 한화
index : 36/2670, 보해양조
index : 37/2670, 유니온
index : 38

In [15]:
# 모바일 내에서 검색한 뉴스들의 제목 & 내용 가져옴
def mobile_news_crawling(keyword_list, day):
    result_object = {}  # 결과를 저장할 객체

    for keyword in keyword_list:
        url = f"https://m.search.naver.com/search.naver?where=m_news&query={keyword}&sm=mtb_opt&sort=0&photo=0&field=0&pd=3 \
        &ds={day}&de={day}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&is_sug_officeid=0 \
        &office_category=0&service_area=0"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = []
        for item in soup.select('.list_news .bx'):
            
            press = item.select_one('a.info.press').get_text().replace(' 선정', '').strip()
            anchor = item.select_one('a.news_tit')
    
            title = anchor.get_text().strip()
            url = anchor['href']
            dsc = item.select_one('.news_dsc').get_text().strip()
            result.append({
                'press': press,
                'title': title,
                'url': url,
                'dsc': dsc,
            })

        result_object[keyword] = result  # 키워드를 키로 사용하여 결과 저장
    
    return result_object  # 객체 반환


In [ ]:
from datetime import datetime
current_date_formatted = datetime.now().strftime("%Y.%m.%d")

2024.06.20


In [14]:
import requests
#네이버 모바일에서의 뉴스 포맷을 크롤링
def get_mobile_dsc(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    result = []
    for item in soup.find('article', id='dic_area'):
        result.append(item.get_text(strip=True))
    
    return result
    
mobile_result = mobile_news_crawling(['삼성전자'], "2024.06.01")
get_mobile_dsc("https://n.news.naver.com/article/262/0000017491?sid=101")

print()

In [12]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

In [9]:
def fetch_page(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"Error fetching page: {url}", e)
        return None

def decode_html(response):
    content_type = response.headers.get("content-type")
    charset = "UTF-8"
    if "charset=" in content_type:
        charset = content_type.split("charset=")[1]
    return response.content.decode(charset)

def fetch_main(url, start_page, end_page):
    response = fetch_page(url)
    if response:
        data = decode_html(response)
        soup = BeautifulSoup(data, 'html.parser')
        title = "뉴스 리스트영역"
        name = "news"; 
        
        news_src = soup.find('iframe', {'title': title, 'name': name})['src']
        news_src = news_src.replace("&page=&clusterId=","")

        results = []
        for page in range(start_page, end_page + 1):
            page_url = f"{base_url}{news_src}&page={page}"
            results.extend(fetch_news(page_url, end_page))
            
        return results
    
#url 하나 당 뉴스 쫘라락 있는데 그거 가져오기
def fetch_news(url, end_page): 
    response = fetch_page(url)
    if response:
        data = decode_html(response)
        soup = BeautifulSoup(data, 'html.parser')
        results = []
        table_rows = soup.select("table.type5 tr")
            
        for row in table_rows:
            cells = row.find_all('td')
            if len(cells) == 3:
                title = cells[0].text.strip()
                link = base_url + cells[0].a['href']
                provider = cells[1].text.strip()
                date = cells[2].text.strip()
                results.append({
                    "title": title,
                    "link": link,
                    "provider": provider,
                    "date": date,
                })
        return results
    return []



In [11]:
code = "005930"
url = f"https://finance.naver.com/item/news.nhn?code={code}"

returned_data = fetch_main(url, 1, 5)
for data in returned_data :
    print(data['title'])
    print(data['link'])

NameError: name 'base_url' is not defined

In [1]:
import socket

def get_local_ip():
    try:
        hostname = socket.gethostname()
        local_ip = socket.gethostbyname(hostname)
        return local_ip
    except socket.error as e:
        print(f"Error fetching local IP: {e}")
        return None

local_ip = get_local_ip()
print(f"Local IP: {local_ip}")

Local IP: 172.31.13.91


In [ ]:
from transformers import pipeline
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import joblib
# 감정 분석 파이프라인 로드

def get_sentiment(text):
    result = sentiment_model(text)
    for elem in result :
        if elem['label'] == "LABEL_1" :
            elem['label'] = "POSITIVE"
        else :
            elem['label'] = "NEGATIVE"
        elem['score'] = round(elem['score'],5)

    print(f"Sentiment analysis result: {result}")
    return result


app = Flask(__name__)

@app.route('/sentiment', methods=['POST'])
def predict():
    global sentiment_model
    try:
        data = request.json['data']
        sentiment_result = get_sentiment(data)
        
        print(sentiment_result)
        return jsonify({'sentiment analysis': sentiment_result })
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == "__main__":
    global sentiment_model
    sentiment_model = pipeline(model="WhitePeak/bert-base-cased-Korean-sentiment")
    # print(f'Model loaded from {model_path}')

    from waitress import serve
    serve(app, host="0.0.0.0", port=8080)

Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.98365}]
[{'label': 'NEGATIVE', 'score': 0.98365}]
